This notebook gives an overview of the workflow used to carry out different quenching corrections on the GNATS glider data. All correction methods are based on [Thomalla et al (2018)](https://aslopubs.onlinelibrary.wiley.com/doi/10.1002/lom3.10234), but with slight adaptions.

The quenching correction options:

1. Night: take the mean ($N_m$) or the first and last profiles ($N_{fl}$)
2. Euphotic zone: limit quenching depth to be within Zeu ($Zeu_y$) or not ($Zeu_n$) or based on a hard limit ($Zeu_h$)

If we do all the combinations of the above we end up with 6 different quenching correction workflows:

1. MZy: $N_m\rightarrow Zeu_y$
1. MZn: $N_m\rightarrow Zeu_n$
1. MZh: $N_m\rightarrow Zeu_h$
1. FLZy: $N_{fl}\rightarrow Zeu_y$
1. FLZy: $N_{fl}\rightarrow Zeu_n$
1. FLZh: $N_{fl}\rightarrow Zeu_h$

To do this analysis, we'll make use of the `quenchingFunctions` module which contains a lot of the functions needed.

# Initialisation

First modules:

In [1]:
import xarray as xr
import numpy as np
import os
import re
import quenchingFunctions as qf
import warnings

warnings.filterwarnings('ignore', message='Mean of empty slice')

Then a list of the glider data files:

In [2]:
parentDir = '../../../data/mission-netcdf/darkCountCorrected/'
files = [f for f in os.listdir(parentDir) 
         if os.path.isfile(os.path.join(parentDir,f))]
gFiles = [parentDir + ff for ff in files[:-1]]

We're also going to need the euphotic depth, which is stored in the `secondaryParams.nc` file

In [5]:
secParams = xr.open_dataset('../secondaryParams-v2.nc')

And we're going to need to iterate through the different correction approaches. In the `quenchingFunctions` module, the euphotic zone conditions are written as different functions, and depending on the night-time interpolation we're going to need different reference arrays. Hence, we are going to need to call different functions, depending on the correction approach. 

Function Reference Lists:

In [6]:
fnights = {'M' : qf.refArrayMeanNights, 'FL' : qf.refArrayEdgeNights}
fzq = {'Zn' : qf.findQuenchingDepthSC, 
       'Zy' : qf.findQuenchingDepthSCInEZ,
       'Zh' : qf.findQuenchingDepthSCupper}
directions = ['east', 'west']

Correction method list:

In [7]:
chlcorrmethod = ['MZn', 'MZy', 'MZh',
                'FLZn', 'FLZy', 'FLZh', 'nocorr']

Adapting the above such that the corrected data are stored in a dataset with three dimensions as described above:

# Applying quenching correction methods to all missions

Here we'll apply the different quenching corrections to all the glider missions. The output will be two netCDF files (xarray datasets):

1. The corrected chlorophyll fluorescence data for all the missions, with all the different correction methods, so we can evaluated the different methods against each other.
2. The calculated quenching depths.

In [9]:
# preallocating final dataset
# need to get the coords from one of the glider mission files
# but doesn't matter which, because they all have the same coords
gdata = xr.open_dataset(gFiles[0])
allCorrectedChl = xr.Dataset(coords = {
        "zbin": ("zbin", gdata.zbin),
        "lonbin": ("lonbin", gdata.lonbin),
        "chlcorrmethod": ("chlcorrmethod", chlcorrmethod)})
allQZ = xr.Dataset(coords = {
        "lonbin": ("lonbin", gdata.lonbin),
        "chlcorrmethod": ("chlcorrmethod", chlcorrmethod)})

# looping through glider missions
for ix,gf in enumerate(gFiles):
    # file name manipulations for IDs
    gid = re.split('/',gf)[-1][:-3]
    g0 = gid[0]
    gE = re.search('[0-9]*\.?[0-9]',gid).group()
    
    print(gid)
    
    # import data
    gdata = xr.open_dataset(gf)
    
    # finding nights
    nightData = qf.findingNights(gdata)
    
    # preallocating
    missionCorrected = {}
    missionQZ = {}
    
    # quenching correction workflow
    for dd in directions:
        chlCorrected = []
        quenchDepths = []
        for nkey, fn in fnights.items(): #for each night condition
            for zkey, fz in fzq.items(): #for each quenching & euphotic depth condition

                zeu = secParams.sel(mission = gid+'_'+dd).Zeu
                refArrays = fn(nightData[dd],gdata,dd,gdata.lonbin)
                qdepths = fz(gdata, refArrays, dd, zeu)
                quenchDepths +=[qdepths]
                correctedChl = qf.correctQuenching(gdata, refArrays, dd, qdepths)              
                chlCorrected += [correctedChl.assign_coords(chlcorrmethod=nkey+zkey).expand_dims('chlcorrmethod')]

        # storing all the corrected data in a data array temporarily
        missionCorrected[dd] = xr.concat(chlCorrected, dim = 'chlcorrmethod')
        missionQZ[dd] = xr.DataArray(np.stack(quenchDepths + [np.ones(quenchDepths[0].shape)*np.nan]),coords=[chlcorrmethod,gdata.lonbin],dims=['chlcorrmethod','lonbin'])

    # storing the mission data in the final dataset
    # this is done in a separate loop to ensure all the chlcorrmethod coordinates
    # align correctly when combining into the main dataset
    for dd in directions:
        rawchl = gdata['chlfl_'+dd].assign_coords(chlcorrmethod = 'nocorr').expand_dims('chlcorrmethod')
        allCorrectedChl[g0+gE+'_'+dd] = missionCorrected[dd].combine_first(rawchl)
        allQZ[g0+gE+'_'+dd] = missionQZ[dd]
    

henry-mission18
grampus-mission1
grampus-mission2.1
grampus-mission2
grampus-mission3
grampus-mission4
grampus-mission5
grampus-mission6
grampus-mission7
grampus-mission8
henry-mission1
henry-mission2
henry-mission3
henry-mission4
henry-mission5
henry-mission6
henry-mission7.1
henry-mission7
henry-mission8
henry-mission9
henry-mission10
henry-mission11
henry-mission12
henry-mission13
henry-mission14
henry-mission15
henry-mission16


Outputing data:

In [10]:
allCorrectedChl.to_netcdf('../../../data/quenching-evaluation-netcdf/quenchingCorrectionTests.nc')
allQZ.to_netcdf('../../../data/quenching-evaluation-netcdf/quenchingDepths.nc')